# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [2]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40506 entries, 0 to 40505
Columns: 464 entries, Wip Line_Dam to target
dtypes: float64(350), int64(77), object(37)
memory usage: 143.4+ MB


In [4]:
train_data.shape[0]

40506

In [5]:
df = train_data.copy()

In [6]:
# 각 열별 결측치 개수 확인
missing_values_count = (df.isnull().sum())
missing_values_count = missing_values_count/df.shape[0] 
missing_values_ratio = missing_values_count * 100
print(missing_values_ratio)

Wip Line_Dam                       0.0
Process Desc._Dam                  0.0
Equipment_Dam                      0.0
Model.Suffix_Dam                   0.0
Workorder_Dam                      0.0
                                 ...  
Receip No Judge Value_Fill2      100.0
WorkMode Collect Result_Fill2      0.0
WorkMode Unit Time_Fill2         100.0
WorkMode Judge Value_Fill2       100.0
target                             0.0
Length: 464, dtype: float64


In [7]:
bins = [0, 20, 40, 60, 80, 100]
missing_values_counts = pd.cut(missing_values_ratio, bins=bins).value_counts().sort_index()

print(missing_values_counts)

(0, 20]        0
(20, 40]       3
(40, 60]       0
(60, 80]       5
(80, 100]    278
Name: count, dtype: int64


(20, 40]에 대해서는 결측치를 채우는 과정을 수행한다.

In [17]:
# 결측치 비율이 80 이상인 열 삭제
threshold = 80
df= df.drop(columns=missing_values_ratio[missing_values_ratio >= threshold].index)

df.info

<bound method DataFrame.info of       Wip Line_Dam Process Desc._Dam     Equipment_Dam Model.Suffix_Dam  \
0          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
1          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334505   
2          IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
3          IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
4          IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
...            ...               ...               ...              ...   
40501      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
40502      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40503      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
40504      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
40505      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   

      Workorder_Dam  Insp. Seq No._Dam Insp Judge Code_Dam  \
0    

칼럼들을 각 공정 단계 별로 구별한다.

In [36]:
# "Process Desc"를 포함하는 열의 인덱스 찾기
process_columns = [col for col in df.columns if "Process Desc" in col]
process_indices = [df.columns.get_loc(col) for col in process_columns]

# 새로운 DataFrame을 저장할 딕셔너리 초기화
dfs = {}

# Process Desc에 따라 DataFrame을 나누어 저장
for i, start_idx in enumerate(process_indices):
    end_idx = process_indices[i + 1]-1 if i + 1 < len(process_indices) else len(df.columns)-1
    
    # 각 Process Desc와 그에 해당하는 환경 변수들을 포함하는 DataFrame 생성
    relevant_columns = df.columns[start_idx:end_idx]
    dfs[f'df{i+1}'] = df[relevant_columns]

# 나눠진 DataFrame들을 확인
for name, df_subset in dfs.items():
    print(f'{name}:')
    print(df_subset)
    print('\n')

df1:
      Process Desc._Dam     Equipment_Dam Model.Suffix_Dam Workorder_Dam  \
0         Dam Dispenser  Dam dispenser #1      AJX75334505    4F1XA938-1   
1         Dam Dispenser  Dam dispenser #1      AJX75334505    3KPM0016-2   
2         Dam Dispenser  Dam dispenser #2      AJX75334501    4E1X9167-1   
3         Dam Dispenser  Dam dispenser #2      AJX75334501    3K1X0057-1   
4         Dam Dispenser  Dam dispenser #1      AJX75334501    3HPM0007-1   
...                 ...               ...              ...           ...   
40501     Dam Dispenser  Dam dispenser #1      AJX75334501    3J1XF434-2   
40502     Dam Dispenser  Dam dispenser #2      AJX75334501    4E1XC796-1   
40503     Dam Dispenser  Dam dispenser #1      AJX75334501    4C1XD438-1   
40504     Dam Dispenser  Dam dispenser #2      AJX75334501    3I1XA258-1   
40505     Dam Dispenser  Dam dispenser #1      AJX75334501    3G1XA501-1   

       Insp. Seq No._Dam Insp Judge Code_Dam  \
0                      1          

In [37]:
df1 = dfs['df1']
df2 = dfs['df2']
df3 = dfs['df3']
df4 = dfs['df4']

In [40]:
target = df['target']

In [23]:
df1

,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,...,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,WorkMode Collect Result_Dam
0,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,100,...,5800,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000
1,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,70,...,4000,4000,4000,4000,4000,4000,0,0.000,0.000,7.000
2,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,85,...,5800,5800,5800,5800,5800,5800,5800,0.012,-0.022,0.003
3,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,70,...,4000,4000,4000,4000,4000,4000,0,0.000,0.000,7.000
4,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,70,...,9000,9000,9000,9000,9000,9000,0,0.000,0.000,7.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,70,...,5000,5000,5000,5000,5000,5000,0,0.000,0.000,7.000
40502,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,100,...,5800,5800,5800,5800,5800,5800,5800,-0.019,-0.021,-0.118
40503,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,100,...,6000,6000,6000,6000,6000,6000,6000,0.000,0.000,0.000
40504,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,70,...,9000,9000,9000,9000,9000,9000,0,0.000,0.000,7.000


In [27]:
df2

,Process Desc._AutoClave,Equipment_AutoClave,Model.Suffix_AutoClave,Workorder_AutoClave,Insp. Seq No._AutoClave,Insp Judge Code_AutoClave,1st Pressure Collect Result_AutoClave,1st Pressure 1st Pressure Unit Time_AutoClave,1st Pressure Judge Value_AutoClave,2nd Pressure Collect Result_AutoClave,2nd Pressure Unit Time_AutoClave,2nd Pressure Judge Value_AutoClave,3rd Pressure Collect Result_AutoClave,3rd Pressure Unit Time_AutoClave,3rd Pressure Judge Value_AutoClave,Chamber Temp. Collect Result_AutoClave,Chamber Temp. Unit Time_AutoClave,Chamber Temp. Judge Value_AutoClave,GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave,GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave
0,Auto Clave Out,Auto Clave Out,AJX75334505,4F1XA938-1,1,OK,0.312,240,OK,0.493,1,OK,0.499,120,OK,54,361,OK,NaN,NaN
1,Auto Clave Out,Auto Clave Out,AJX75334505,3KPM0016-2,1,OK,0.311,241,OK,0.311,121,OK,0.498,121,OK,54,483,OK,NaN,NaN
2,Auto Clave Out,Auto Clave Out,AJX75334501,4E1X9167-1,1,OK,0.315,241,OK,0.490,1,OK,0.498,121,OK,56,363,OK,OK,OK
3,Auto Clave Out,Auto Clave Out,AJX75334501,3K1X0057-1,1,OK,0.299,241,OK,0.300,121,OK,0.500,121,OK,54,483,OK,NaN,NaN
4,Auto Clave Out,Auto Clave Out,AJX75334501,3HPM0007-1,1,OK,0.302,240,OK,0.302,120,OK,0.498,120,OK,47,480,NG,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Auto Clave Out,Auto Clave Out,AJX75334501,3J1XF434-2,1,OK,0.300,241,OK,0.300,121,OK,0.500,121,OK,54,483,OK,NaN,NaN
40502,Auto Clave Out,Auto Clave Out,AJX75334501,4E1XC796-1,1,OK,0.305,240,OK,0.493,1,OK,0.498,120,OK,49,361,NG,NaN,NaN
40503,Auto Clave Out,Auto Clave Out,AJX75334501,4C1XD438-1,1,OK,0.309,241,OK,0.490,1,OK,0.498,121,OK,48,363,NG,OK,OK
40504,Auto Clave Out,Auto Clave Out,AJX75334501,3I1XA258-1,1,OK,0.302,241,OK,0.303,121,OK,0.499,121,OK,54,483,OK,NaN,NaN


상수 변수 탐지 : 모든 항이 동일한 상수 변수의 경우 가치가 존재하지 않는다.

In [30]:
# 상수 열 찾기
constant_columns1 = [col for col in df1.columns if df1[col].nunique() == 1]
constant_columns1

['Process Desc._Dam',
 'Insp. Seq No._Dam',
 'Insp Judge Code_Dam',
 'CURE STANDBY POSITION X Collect Result_Dam',
 'CURE STANDBY POSITION Z Collect Result_Dam',
 'CURE STANDBY POSITION Θ Collect Result_Dam',
 'CURE START POSITION Z Collect Result_Dam',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Dam']

In [35]:
# 상수 열 삭제
df1_cleaned = df1.drop(columns=constant_columns)
print(df1_cleaned.shape[1])
print(df1.shape[1])

69
77


In [44]:
def remove_constant_columns(df):
    """
    DataFrame에서 상수 열(모든 값이 동일한 열)을 찾아 삭제하고, 삭제된 열의 이름을 반환합니다.
    
    Parameters:
    df (pd.DataFrame): 상수 열을 삭제할 DataFrame

    Returns:
    tuple: (상수 열이 제거된 DataFrame, 삭제된 상수 열의 리스트)
    """
    # 상수 열 찾기
    constant_columns = [col for col in df.columns if df[col].nunique() == 1]
    
    # 상수 열 출력
    print("삭제된 상수 열:", constant_columns)
    
    # 상수 열 삭제
    df_cleaned = df.drop(columns=constant_columns)
    
    return df_cleaned, constant_columns

In [47]:
df2_cleaned, constant_columns2 = remove_constant_columns(df2)
df3_cleaned, constant_columns3 = remove_constant_columns(df3)
df4_cleaned, constant_columns4 = remove_constant_columns(df4)

삭제된 상수 열: ['Process Desc._AutoClave', 'Equipment_AutoClave', 'Insp. Seq No._AutoClave', 'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Collect Result_AutoClave', 'GMES_ORIGIN_INSP_JUDGE_CODE Judge Value_AutoClave']
삭제된 상수 열: ['Process Desc._Fill1', 'Insp. Seq No._Fill1', 'Insp Judge Code_Fill1', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Judge Value_Fill1']
삭제된 상수 열: ['Process Desc._Fill2', 'Insp. Seq No._Fill2', 'Insp Judge Code_Fill2', 'CURE END POSITION Θ Collect Result_Fill2', 'CURE STANDBY POSITION X Collect Result_Fill2', 'CURE STANDBY POSITION Θ Collect Result_Fill2', 'CURE START POSITION Θ Collect Result_Fill2', 'DISCHARGED SPEED OF RESIN Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill2', 'Dispense Volume(Stage1) Colle

In [ ]:
df

### 언더 샘플링


데이타 불균형을 해결하기 위해 언더 샘플링을 진행합니다.


In [3]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

### 데이터 분할


In [4]:
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.3,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 1645, AbNormal: 1645 ratio: 1.0
  Total: Normal: 705, AbNormal: 705 ratio: 1.0


## 3. 모델 학습


### 모델 정의


In [5]:
model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습


In [6]:
features = []

for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

train_x = df_train[features]
train_y = df_train["target"]

model.fit(train_x, train_y)

RandomForestClassifier(random_state=110)

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [7]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

In [8]:
df_test_x = test_data[features]

for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [9]:
test_pred = model.predict(df_test_x)
test_pred

array(['AbNormal', 'Normal', 'AbNormal', ..., 'Normal', 'Normal',
       'AbNormal'], dtype=object)

### 제출 파일 작성


In [10]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
